# Instalando ambiente de desenvolvimento

Para tal, mude o status do campo abaixo para code e execute

In [1]:
# !pip install numpy
# !pip install pandas
# !pip install multipledispatch
# !pip install yfinance
# !pip install gym
# !pip install stable-baselines gym-anytrading gym
# !pip install -U scikit-learn

# Lista de frameworks e bibliotecas

$$ \frac {x_1}{x_2} $$

In [7]:
import numpy as np
import pandas as pd
import yfinance as yf # https://github.com/ranaroussi/yfinance
from multipledispatch import dispatch
from datetime import datetime, timedelta
import gym
import os

# Environment Variable

In [3]:
STOCK_NAME = "USDBRL=X"
ASSETS_URL = "assets/"
FILE_NAME = "usd2brl.csv"
FILE_URL = ASSETS_URL + FILE_NAME
LOG_PATH = "logs/"
PERIOD = "7d" # 7 days
INTERVAL = "1m" # 1 minute

# Obtendo dados

In [7]:
class Database(yf.Ticker):
    STOCK_NAME = "USDBRL=X"
    PERIOD = "7d" # 7 days
    INTERVAL = "1m" # 1 minute
    
    def __init__(self, stock_name: str):
        self.STOCK_NAME = stock_name
        
    def connect(self):
        try:
            return yf.Ticker(self.STOCK_NAME)
        except OverflowError as e:
            raise e
    
    def set_period(self, period: str):
        self.PERIOD = period
        
    def set_interval(self, interval: str):
        self.INTERVAL = interval
        
    @dispatch()
    def get_data(self) -> pd.DataFrame:
        try:
            database = self.connect()
            dataframe =  pd.DataFrame(database.history(period=self.PERIOD, interval=self.INTERVAL))
            dataframe = dataframe.reset_index()
            try:
                return dataframe[['Date', 'Open','High','Low','Close']]
            except:
                return dataframe[['Datetime',  'Open','High','Low','Close']]
        except:
            pass

    @dispatch(datetime, datetime)
    def get_data(self, start_date: datetime, end_date: datetime):
        try: 
            database = self.connect()
            dataframe = self.database.history(period=self.PERIOD, interval=self.INTERVAL, start=start_date, end=end_date)
            dataframe = dataframe.reset_index()
            try:
                return dataframe[['Date', 'Open','High','Low','Close']]
            except:
                return dataframe[['Datetime',  'Open','High','Low','Close']]
        except :
            pass
    
    def export_csv(self, path: str, file_name: str):
        
        pathfile = path + file_name
        
        dataframe = self.get_data()
        if not os.path.exists(path):
            os.mkdir(path)
        files_present = os.path.isfile(pathfile) 
        if not files_present:
            # loggin.info(f"The file {file_name} is being written")
            dataframe.to_csv(pathfile)
            print(dataframe)
        else:
            # logging.info("The file already exists, it will be deleted and rewritten")
            os.remove(pathfile)
            dataframe.to_csv(pathfile)

In [8]:
data = Database(STOCK_NAME)
data.export_csv(ASSETS_URL, FILE_NAME)

# Carregando dados CSV

In [4]:
dataset = pd.read_csv(ASSETS_URL + FILE_NAME)
dataset = dataset.iloc[:, 1:] 
dataset

,Datetime,Open,High,Low,Close
0,2022-02-02 00:00:00+00:00,5.2735,5.2735,5.2735,5.2735
1,2022-02-02 00:30:00+00:00,5.2735,5.2735,5.2735,5.2735
2,2022-02-02 00:31:00+00:00,5.2735,5.2735,5.2735,5.2735
3,2022-02-02 00:32:00+00:00,5.2735,5.2735,5.2735,5.2735
4,2022-02-02 00:33:00+00:00,5.2735,5.2735,5.2735,5.2735
...,...,...,...,...,...
4252,2022-02-10 00:54:00+00:00,5.2176,5.2176,5.2176,5.2176
4253,2022-02-10 00:55:00+00:00,5.2176,5.2176,5.2176,5.2176
4254,2022-02-10 01:10:00+00:00,5.2176,5.2176,5.2176,5.2176
4255,2022-02-10 01:30:00+00:00,5.2176,5.2176,5.2176,5.2176


In [9]:
# Gym stuff
import gym
import gym_anytrading 

# stable baselines
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import A2C

# processing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

https://www.youtube.com/watch?v=D9sU1hLT0QY&ab_channel=NicholasRenotte


ModuleNotFoundError: No module named 'gym_anytrading'